# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven. 

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**. 

In [72]:
import numpy as np
import tensorflow as tf

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [73]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [74]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de muestras de prueba

In [75]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [76]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [77]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [78]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [79]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [80]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [81]:
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjunto de validación y prueba, no se necesita barajearlos

In [82]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprenzaje profundo, casi siempre solo lo imaginamos.  Asi que esta vez, hagámoslo.  :)

In [83]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  No es una necesidad!

In [84]:
tamanio_capa_escondida = 50
# tamanio_capa_escondida = 200
# tamanio_capa_escondida = 300


# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None), o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  output = activation(dot(entrada, peso) + sesgo).  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [85]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4ta capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5ta capa escondida
    
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida
    
    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

c:\Users\Alejandro\Documents\GitHub\LAB-02-DT\lab02_ds\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [86]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [87]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 4ms/step - accuracy: 0.8764 - loss: 0.4400 - val_accuracy: 0.9417 - val_loss: 0.2073
Epoch 2/5
500/500 - 1s - 2ms/step - accuracy: 0.9414 - loss: 0.1993 - val_accuracy: 0.9548 - val_loss: 0.1629
Epoch 3/5
500/500 - 1s - 2ms/step - accuracy: 0.9539 - loss: 0.1543 - val_accuracy: 0.9609 - val_loss: 0.1358
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9634 - loss: 0.1230 - val_accuracy: 0.9648 - val_loss: 0.1180
Epoch 5/5
500/500 - 2s - 3ms/step - accuracy: 0.9688 - loss: 0.1029 - val_accuracy: 0.9658 - val_loss: 0.1208


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. Nunca debe probarse el modelo antes de haber completamente ajustado el modelo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [88]:
perdida_prueba, precision_entrenamiento = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9521 - loss: 25.5391


In [89]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_entrenamiento * 100.))

Pérdida de prueba: 21.49. Precisión de prueba: 95.88%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.

# Encontrando el ancho ideal

In [90]:
# # Lista de tamaños de capas escondidas para probar
# tamanios_capa_escondida = [50, 100, 150, 200, 250, 300]

# # Variable para guardar el mejor tamaño de capa escondida
# mejor_tamanio = None
# mejor_precision_validacion = 0

# # Iterar sobre los diferentes tamaños de capa escondida
# for tamanio in tamanios_capa_escondida:
#     # Definir el modelo con el tamaño de capa escondida actual
#     modelo = tf.keras.Sequential([
#         tf.keras.layers.Flatten(input_shape=(28, 28)),
#         tf.keras.layers.Dense(tamanio, activation='relu'),
#         tf.keras.layers.Dense(tamanio, activation='relu'),
#         tf.keras.layers.Dense(tamanio_salida, activation='softmax')
#     ])

#     # Compilar el modelo
#     modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     # Entrenar el modelo
#     historia = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=0)

#     # Obtener la precisión de validación final
#     precision_validacion = historia.history['val_accuracy'][-1]

#     # Imprimir la precisión de validación para este tamaño de capa escondida
#     print(f'Tamaño de capa escondida: {tamanio}, Precisión de validación: {precision_validacion:.2f}%')

#     # Verificar si este es el mejor tamaño de capa escondida encontrado hasta ahora
#     if precision_validacion > mejor_precision_validacion:
#         mejor_precision_validacion = precision_validacion
#         mejor_tamanio = tamanio

# print(f'\nEl mejor tamaño de capa escondida es {mejor_tamanio} con una precisión de validación de {mejor_precision_validacion:.2f}%')


# Probando diferentes tamaños de tanda

Probamos con tamaños de tanda de 10,000 y de 1.

In [91]:
import pandas as pd
import time

*Esta celda tarda aprox. 6 min en ejecutarse...

In [92]:
# Lista para guardar los resultados
results = []

# Lista de tamaños de tanda
tamanios_tanda = [10000, 1]

# Definir el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(tamanio_salida, activation='softmax')
])
    
# Compilar el modelo
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
# Iterar sobre los diferentes tamaños de tanda
for tamanio in tamanios_tanda:
    # Reeestablecer los conjuntos de datos
    datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
    datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
    datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))
    
    # Dividir en tandas los conjuntos de datos
    datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(tamanio)  # Barajear datos de entrenamiento
    datos_validacion = datos_validacion.batch(tamanio)
    datos_prueba = datos_prueba.batch(tamanio)
    
    # Entrenar el modelo con los datos divididos
    print(f"Entrenando modelo con tamaño de tanda '{tamanio}'...", end=" ")
    start = time.time()
    historia = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=0)
    end = time.time()
    tiempo_entrenamiento = end - start
    print("Listo!")

    # Obtener la pérdida
    perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

    # Obtener la precisión de prueba final
    precision_entrenamiento = historia.history['accuracy'][-1]

    # Obtener la precisión de validación final
    precision_validacion = historia.history['val_accuracy'][-1]

    # Guardar los resultados
    results.append({
        "Tamaño de tanda": tamanio,
        "Precisión de entrenamiento": "{:.2f}%".format(precision_entrenamiento * 100),
        "Precisión de prueba": "{:.2f}%".format(precision_prueba * 100),
        "Precisión de validación": "{:.2f}%".format(precision_validacion * 100),
        "Tiempo de entrenamiento (s)": "{:.2f}".format(tiempo_entrenamiento)
    })


Entrenando modelo con tamaño de tanda '10000'... Listo!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.7928 - loss: 38.4695
Entrenando modelo con tamaño de tanda '1'... Listo!
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 9s 927us/step - accuracy: 0.9510 - loss: 61.4987


In [93]:
# Mostrar resultados como dataframe
df = pd.DataFrame(results)
df

,Tamaño de tanda,Precisión de entrenamiento,Precisión de prueba,Precisión de validación,Tiempo de entrenamiento (s)
0,10000,74.10%,79.28%,80.69%,3.20
1,1,96.76%,95.89%,96.77%,336.49


# Probando diferentes tasas de apredizaje

In [94]:
from tensorflow.keras.optimizers import Adam

In [95]:
# Lista para guardar los resultados
results = []

# Lista de tasas de aprendizaje para probar
tasas_aprendizaje = [0.0001, 0.02]

# Definir el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(tamanio_salida, activation='softmax')
])
    
# Reeestablecer los conjuntos de datos
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

# Dividir en tandas los conjuntos de datos con el tamaño original
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)  # Barajear datos de entrenamiento
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)
    
# Iterar sobre las diferentes tasas de aprendizaje
for tasa in tasas_aprendizaje:
    # Crear el optimizador
    optimizador = Adam(learning_rate=tasa)
    
    # Compilar el modelo con el optimizador
    modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Entrenar el modelo con los datos divididos
    print(f"Entrenando modelo con tasa de apredizaje '{tasa}'...", end=" ")
    start = time.time()
    historia = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=0)
    end = time.time()
    tiempo_entrenamiento = end - start
    print("Listo!")

    # Obtener la pérdida
    perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

    # Obtener la precisión de prueba final
    precision_entrenamiento = historia.history['accuracy'][-1]

    # Obtener la precisión de validación final
    precision_validacion = historia.history['val_accuracy'][-1]

    # Guardar los resultados
    results.append({
        "Tasa de aprendizaje": tasa,
        "Precisión de entrenamiento": "{:.2f}%".format(precision_entrenamiento * 100),
        "Precisión de prueba": "{:.2f}%".format(precision_prueba * 100),
        "Precisión de validación": "{:.2f}%".format(precision_validacion * 100),
        "Tiempo de entrenamiento (s)": "{:.2f}".format(tiempo_entrenamiento)
    })


c:\Users\Alejandro\Documents\GitHub\LAB-02-DT\lab02_ds\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Entrenando modelo con tasa de apredizaje '0.0001'... Listo!
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9095 - loss: 38.3548
Entrenando modelo con tasa de apredizaje '0.02'... Listo!
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9383 - loss: 43.6595


In [96]:
# Mostrar resultados como dataframe
df = pd.DataFrame(results)
df

,Tasa de aprendizaje,Precisión de entrenamiento,Precisión de prueba,Precisión de validación,Tiempo de entrenamiento (s)
0,0.0001,92.03%,92.31%,92.95%,6.75
1,0.0200,95.51%,94.58%,95.29%,8.08


# Mejorando la precisión de validación
Se modificarán los parámetros y la composición del modelo con el fin de obtener una precisión de validación mayor o igual a 98.5%.

In [210]:
# Establecer parámetros
TAMANIO_TANDA = 600
TAMANIO_CAPA_ESCONDIDA = 300
TASA_APRENDIZAJE = 0.02
NUMERO_EPOCAS = 10

In [211]:
# Reestablecer los conjuntos de datos
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

# Dividir en tandas los conjuntos de datos
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)  # Barajear datos de entrenamiento
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

# Definimos la nueva estructura del modelo
modelo = tf.keras.Sequential([
    tf.keras.Input(shape=(28, 28)), # capas de entrada
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(TAMANIO_CAPA_ESCONDIDA, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(TAMANIO_CAPA_ESCONDIDA, activation='sigmoid'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

# Crear el optimizador con una mayor tasa de aprendizaje
optimizador = Adam(learning_rate=TASA_APRENDIZAJE)

# Compilamos el modelo utilizando la tasa de aprendizaje por defecto (0.001)
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Crear variable para almacenar los resultados
results = []

# Entrenar el modelo
print(f"Entrenando modelo...")
start = time.time()
historia = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=1)
end = time.time()
tiempo_entrenamiento = end - start
print("Listo!")

# Obtener la pérdida
print("Evaluando modelo...")
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

# Obtener la precisión de prueba final
precision_entrenamiento = historia.history['accuracy'][-1]

# Obtener la precisión de validación final
precision_validacion = historia.history['val_accuracy'][-1]

# Guardar los resultados
results.append({
    "Precisión de entrenamiento": "{:.2f}%".format(precision_entrenamiento * 100),
    "Precisión de prueba": "{:.2f}%".format(precision_prueba * 100),
    "Precisión de validación": "{:.2f}%".format(precision_validacion * 100),
    "Tiempo de entrenamiento (s)": "{:.2f}".format(tiempo_entrenamiento)
})
print("Listo!")
    

Entrenando modelo...
Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.4576 - loss: 1.9719 - val_accuracy: 0.9460 - val_loss: 0.1937
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9480 - loss: 0.1722 - val_accuracy: 0.9626 - val_loss: 0.1235
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9721 - loss: 0.0945 - val_accuracy: 0.9683 - val_loss: 0.1055
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9790 - loss: 0.0685 - val_accuracy: 0.9675 - val_loss: 0.1028
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9820 - loss: 0.0575 - val_accuracy: 0.9700 - val_loss: 0.1003
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9870 - loss: 0.0427 - val_accuracy: 0.9726 - val_loss: 0.0939
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9895 - loss: 0.0325 - val_accuracy: 0.9754 - val_loss: 0.0887
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9935 - loss: 0.0222 - val

In [212]:
# Mostrar resultados como dataframe
df = pd.DataFrame(results)
df

,Precisión de entrenamiento,Precisión de prueba,Precisión de validación,Tiempo de entrenamiento (s)
0,99.15%,96.62%,97.40%,15.64
